### Inventory of stock items, not managed for a month.

In [2]:
import pandas as pd

In [3]:
# gsshop=pd.read_pickle('data/GSS_SELL.pk')
# oliveYoung=pd.read_pickle('data/OLI_SELL.pk')
# hmall=pd.read_pickle('data/HMA_SELL.pk')
gsshop=pd.read_pickle('data/GSS0713.pk')
oliveYoung=pd.read_pickle('data/OLI0713.pk')
hmall=pd.read_pickle('data/HMA0713.pk')


In [4]:
data = oliveYoung;
# data = gsshop;
# data = hmall;
print(data.shape)
data.head(1)

(263808, 18)


,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
0,0,16,16,-1560235313357406774,8758383,2018-07-13 13:07:00,2018-06-02,FILTER ALGO,2018-06-02,A000000000638,불가리 쁘띠마망 EDT 100ml,http://www.oliveyoung.co.kr/store/goods/getGoo...,69000,불가리,4,향수>여성향수,LVMH,0


## SELL_AMOUNT == 0 찾기

In [54]:
Q_SALES = data.groupby('ITEM_ID').SELL_AMOUNT.sum().reset_index()
Q_SALES_ZERO = Q_SALES[Q_SALES.SELL_AMOUNT==0]
print(Q_SALES.shape,Q_SALES_ZERO.shape)
Q_SALES_ZERO.head(1)

(7334, 2) (665, 2)


,ITEM_ID,SELL_AMOUNT
74,8757891,0.0


In [55]:
DATA_ZERO = \
data[data.ITEM_ID.isin(Q_SALES_ZERO.ITEM_ID
                      )][['ITEM_ID','STOCK_ID','SELL_AMOUNT','STOCK_AMOUNT','COLLECT_DAY','BRAND','CATE_NAMES']]
DATA_ZERO.shape

(13148, 7)

## STOCK_AMOUNT == 9999 찾기

In [56]:
DATA_ZERO.loc[DATA_ZERO['STOCK_AMOUNT'].isna(),'STOCK_AMOUNT']=0
DATA_ZERO['STOCK_AMOUNT'] = DATA_ZERO['STOCK_AMOUNT'].astype('int')
DATA_ZERO['STOCK_AMOUNT'] = DATA_ZERO['STOCK_AMOUNT'].astype('str')
DATA_ZERO_VALUE_999 = DATA_ZERO[DATA_ZERO.STOCK_AMOUNT.str.contains('^99+$')]
print(DATA_ZERO.shape,DATA_ZERO_VALUE_999.shape) 
#(212026, 6) (15127, 6)  : ^99+$
# (212026, 6) (20354, 6) : ^99+
DATA_ZERO_VALUE_999.head(1)



(13148, 7) (120, 7)


,ITEM_ID,STOCK_ID,SELL_AMOUNT,STOCK_AMOUNT,COLLECT_DAY,BRAND,CATE_NAMES
198305,8759898,7995800085043919441,0,99999,2018-06-02,init,바디/헤어>덴탈케어


In [57]:
DATA_ZERO_VALUE_999.ITEM_ID.nunique() # TOTAL AMOUNT z

4

# [ALL] Inventory by Brand

In [27]:
# data['SITE_NAME']='GSSHOP'
# IVT_GS = data.groupby(['SITE_NAME','BRAND']).ITEM_ID.nunique().reset_index().pivot(
#     index='SITE_NAME',columns='BRAND',values='ITEM_ID')

print(IVT_GS.shape);IVT_GS.head(1)

(1, 5)


BRAND,AP,LG,LOREAL,LVMH,init
SITE_NAME,,,,,
GSSHOP,3823,3386,1303,1895,39213


In [49]:
data['SITE_NAME']='HMALL'
IVT_HML = data.groupby(['SITE_NAME','BRAND']).ITEM_ID.nunique().reset_index().pivot(
    index='SITE_NAME',columns='BRAND',values='ITEM_ID')
print(IVT_HML.shape);IVT_HML.head(1)

(1, 5)


BRAND,AP,LG,LOREAL,LVMH,init
SITE_NAME,,,,,
HMALL,4424,2839,1210,1401,25259


In [58]:
data['SITE_NAME']='OLIVE'
IVT_OLI = data.groupby(['SITE_NAME','BRAND']).ITEM_ID.nunique().reset_index().pivot(
    index='SITE_NAME',columns='BRAND',values='ITEM_ID')
print(IVT_OLI.shape);IVT_OLI.head(1)

(1, 5)


BRAND,AP,LG,LOREAL,LVMH,init
SITE_NAME,,,,,
OLIVE,244,468,193,85,6344


In [71]:
TOTAL_IVT = pd.concat([
    IVT_GS.reset_index(),
    IVT_HML.reset_index(),
    IVT_OLI.reset_index()
])[['SITE_NAME','AP','LG','LOREAL','LVMH']]
TOTAL_IVT.sort_values(by=['SITE_NAME']).set_index('SITE_NAME')

BRAND,AP,LG,LOREAL,LVMH
SITE_NAME,,,,
GSSHOP,3823,3386,1303,1895
HMALL,4424,2839,1210,1401
OLIVE,244,468,193,85


# [999case] Inventory by Brand

In [75]:
print(DATA_ZERO_VALUE_999.ITEM_ID.nunique() )
DATA_ZERO_VALUE_999.head()

4


,ITEM_ID,STOCK_ID,SELL_AMOUNT,STOCK_AMOUNT,COLLECT_DAY,BRAND,CATE_NAMES,SITE_NAME
198305,8759898,7995800085043919441,0,99999,2018-06-02,init,바디/헤어>덴탈케어,OLIVE
198306,8759898,7995800085043919441,0,99999,2018-06-03,init,바디/헤어>덴탈케어,OLIVE
198307,8759898,7995800085043919441,0,99999,2018-06-04,init,바디/헤어>덴탈케어,OLIVE
198308,8759898,7995800085043919441,0,99999,2018-06-05,init,바디/헤어>덴탈케어,OLIVE
198309,8759898,7995800085043919441,0,99999,2018-06-06,init,바디/헤어>덴탈케어,OLIVE


In [62]:
# DATA_ZERO_VALUE_999['SITE_NAME']='GSSHOP'
# DATA_ZERO_VALUE_999['SITE_NAME']='HMALL'
DATA_ZERO_VALUE_999['SITE_NAME']='OLIVE'

c:\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
DATA_ZERO_VALUE_999_pivot= \
DATA_ZERO_VALUE_999.groupby(['SITE_NAME','BRAND']).ITEM_ID.nunique().reset_index().pivot(
    index='SITE_NAME',columns='BRAND',values='ITEM_ID')
DATA_ZERO_VALUE_999_pivot=DATA_ZERO_VALUE_999_pivot.fillna(0)

In [30]:
# DATA_GS = DATA_ZERO_VALUE_999_pivot
print(DATA_GS.shape);DATA_GS.head(2)

(1, 5)


BRAND,AP,LG,LOREAL,LVMH,init
SITE_NAME,,,,,
GSSHOP,205,316,94,442,6092


In [52]:
# DATA_HML = DATA_ZERO_VALUE_999_pivot
print(DATA_HML.shape);DATA_HML.tail(2)

(1, 5)


BRAND,AP,LG,LOREAL,LVMH,init
SITE_NAME,,,,,
HMALL,316,440,36,124,4525


In [64]:
DATA_OLI = DATA_ZERO_VALUE_999_pivot
print(DATA_OLI.shape);DATA_OLI.head(2)

(1, 1)


BRAND,init
SITE_NAME,
OLIVE,4


In [72]:
TOTAL = pd.concat([
    DATA_GS.reset_index(),
    DATA_HML.reset_index(),
    DATA_OLI.reset_index()
])[['SITE_NAME','AP','LG','LOREAL','LVMH']]
TOTAL.sort_values(by=['SITE_NAME']).set_index('SITE_NAME')

,AP,LG,LOREAL,LVMH
SITE_NAME,,,,
GSSHOP,205.0,316.0,94.0,442.0
HMALL,316.0,440.0,36.0,124.0
OLIVE,NaN,NaN,NaN,NaN
